In [3]:
import numpy as np
import pandas as pd
import json
import imageio

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import Callback

from matplotlib import pylab

from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from scikitplot.metrics import plot_confusion_matrix, plot_roc

import neptune.new as neptune
from neptune.new.integrations.tensorflow_keras import NeptuneCallback

import pathlib
import os

print("Current Working Directory " , os.getcwd())
os.chdir("/home/lucas/KNUMxGolemMLHackaton2022")
print("Current Working Directory " , os.getcwd())

from src.utils.data_preprocessing import normalize, preprocessing
from src.utils.load_data import LoadData
from bin.show_im import show_image

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from matplotlib import cm

Current Working Directory  /home/lucas/KNUMxGolemMLHackaton2022/src/notebooks
Current Working Directory  /home/lucas/KNUMxGolemMLHackaton2022


In [ ]:
my_api = "eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIwNWE4OWRjYy1kYjljLTQwMjEtOGQwMC0yMmVkM2FmZGI2N2UifQ=="

params = {"learning_rate": 0.1}

run_title = "resnet50_test"
tags = ["test"]
EPOCH_NR = 5
BATCH_SIZE = 32

In [13]:
# Model

model_nasn = tf.keras.applications.DenseNet201(
    include_top=False,
    weights=None,
    input_tensor=None,
    input_shape=(256, 256, 3),
    pooling=None,
    classes=1000,
    classifier_activation="softmax"
)


model_nasn.compile(
            optimizer='adam',
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy']
)

In [12]:
run = neptune.init(
    project="CogniGeeks/KNUMxGolemMLHackaton2022",
    api_token=my_api
)


NameError: name 'my_api' is not defined

In [3]:
# log params
run["parameters"] = params

# log name and append tags
run["sys/name"] = run_title
run["sys/tags"].add(tags)

# log loss during training
for epoch in range(100):
    run["train/loss"].log(0.99 ** epoch)

class NeptuneLogger(Callback):
    def on_batch_end(self, batch, logs={}):
        for log_name, log_value in logs.items():
            run["batch/{}".format(log_name)].log(log_value)

    def on_epoch_end(self, epoch, logs={}):
        for log_name, log_value in logs.items():
            run["epoch/{}".format(log_name)].log(log_value)

# log params
run["parameters/epoch_nr"] = EPOCH_NR
run["parameters/batch_size"] = BATCH_SIZE

# log name and append tag
run["sys/name"] = "keras-metrics"
run["sys/tags"].add("advanced")


https://app.neptune.ai/common/colab-test-run/e/COL-39262
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


In [6]:
with open('data/reference_images_part1.json') as json_data:
    data = json.load(json_data)

In [7]:
images = pd.DataFrame(data['images'])
annotations = pd.DataFrame(data['annotations'])
categories = pd.DataFrame(data['categories'])

In [8]:
df = load_data('TRAIN')

In [9]:
# pylab.imshow(df['X'][100])

In [14]:
model_nasn.summary()

Model: "densenet201"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_2 (ZeroPadding2  (None, 262, 262, 3)  0          ['input_4[0][0]']                
 D)                                                                                               
                                                                                                  
 conv1/conv (Conv2D)            (None, 128, 128, 64  9408        ['zero_padding2d_2[0][0]']       
                                )                                                       

 conv2_block4_0_relu (Activatio  (None, 64, 64, 160)  0          ['conv2_block4_0_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block4_1_conv (Conv2D)   (None, 64, 64, 128)  20480       ['conv2_block4_0_relu[0][0]']    
                                                                                                  
 conv2_block4_1_bn (BatchNormal  (None, 64, 64, 128)  512        ['conv2_block4_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block4_1_relu (Activatio  (None, 64, 64, 128)  0          ['conv2_block4_1_bn[0][0]']      
 n)                                                                                               
          

                                                                                                  
 conv3_block2_0_relu (Activatio  (None, 32, 32, 160)  0          ['conv3_block2_0_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block2_1_conv (Conv2D)   (None, 32, 32, 128)  20480       ['conv3_block2_0_relu[0][0]']    
                                                                                                  
 conv3_block2_1_bn (BatchNormal  (None, 32, 32, 128)  512        ['conv3_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block2_1_relu (Activatio  (None, 32, 32, 128)  0          ['conv3_block2_1_bn[0][0]']      
 n)       

 ization)                                                                                         
                                                                                                  
 conv3_block6_1_relu (Activatio  (None, 32, 32, 128)  0          ['conv3_block6_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block6_2_conv (Conv2D)   (None, 32, 32, 32)   36864       ['conv3_block6_1_relu[0][0]']    
                                                                                                  
 conv3_block6_concat (Concatena  (None, 32, 32, 320)  0          ['conv3_block5_concat[0][0]',    
 te)                                                              'conv3_block6_2_conv[0][0]']    
                                                                                                  
 conv3_blo

 conv3_block10_concat (Concaten  (None, 32, 32, 448)  0          ['conv3_block9_concat[0][0]',    
 ate)                                                             'conv3_block10_2_conv[0][0]']   
                                                                                                  
 conv3_block11_0_bn (BatchNorma  (None, 32, 32, 448)  1792       ['conv3_block10_concat[0][0]']   
 lization)                                                                                        
                                                                                                  
 conv3_block11_0_relu (Activati  (None, 32, 32, 448)  0          ['conv3_block11_0_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv3_block11_1_conv (Conv2D)  (None, 32, 32, 128)  57344       ['conv3_block11_0_relu[0][0]']   
          

...

 conv5_block15_1_bn (BatchNorma  (None, 8, 8, 128)   512         ['conv5_block15_1_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 conv5_block15_1_relu (Activati  (None, 8, 8, 128)   0           ['conv5_block15_1_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv5_block15_2_conv (Conv2D)  (None, 8, 8, 32)     36864       ['conv5_block15_1_relu[0][0]']   
                                                                                                  
 conv5_block15_concat (Concaten  (None, 8, 8, 1376)  0           ['conv5_block14_concat[0][0]',   
 ate)                                                             'conv5_block15_2_conv[0][0]']   
          

                                                                                                  
 conv5_block19_concat (Concaten  (None, 8, 8, 1504)  0           ['conv5_block18_concat[0][0]',   
 ate)                                                             'conv5_block19_2_conv[0][0]']   
                                                                                                  
 conv5_block20_0_bn (BatchNorma  (None, 8, 8, 1504)  6016        ['conv5_block19_concat[0][0]']   
 lization)                                                                                        
                                                                                                  
 conv5_block20_0_relu (Activati  (None, 8, 8, 1504)  0           ['conv5_block20_0_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv5_blo

 conv5_block24_0_relu (Activati  (None, 8, 8, 1632)  0           ['conv5_block24_0_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv5_block24_1_conv (Conv2D)  (None, 8, 8, 128)    208896      ['conv5_block24_0_relu[0][0]']   
                                                                                                  
 conv5_block24_1_bn (BatchNorma  (None, 8, 8, 128)   512         ['conv5_block24_1_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 conv5_block24_1_relu (Activati  (None, 8, 8, 128)   0           ['conv5_block24_1_bn[0][0]']     
 on)                                                                                              
          

                                                                                                  
 conv5_block28_1_relu (Activati  (None, 8, 8, 128)   0           ['conv5_block28_1_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv5_block28_2_conv (Conv2D)  (None, 8, 8, 32)     36864       ['conv5_block28_1_relu[0][0]']   
                                                                                                  
 conv5_block28_concat (Concaten  (None, 8, 8, 1792)  0           ['conv5_block27_concat[0][0]',   
 ate)                                                             'conv5_block28_2_conv[0][0]']   
                                                                                                  
 conv5_block29_0_bn (BatchNorma  (None, 8, 8, 1792)  7168        ['conv5_block28_concat[0][0]']   
 lization)

 ate)                                                             'conv5_block32_2_conv[0][0]']   
                                                                                                  
 bn (BatchNormalization)        (None, 8, 8, 1920)   7680        ['conv5_block32_concat[0][0]']   
                                                                                                  
 relu (Activation)              (None, 8, 8, 1920)   0           ['bn[0][0]']                     
                                                                                                  
Total params: 18,321,984
Trainable params: 18,092,928
Non-trainable params: 229,056
__________________________________________________________________________________________________


In [ ]:
model = keras.models.Model(inputs=model_nasn.input, outputs=model_nasn.get_layer('global_max_pooling2d_1').output)
# model_nasn.summary()
emb_nasn = model(preprocessing(df)[0])

pca = PCA(n_components=2)

pca.fit(emb_nasn)
pca_proj = pca.transform(emb_nasn)


# Create a two dimensional t-SNE projection of the embeddings
tsne = TSNE(2, verbose=1)
tsne_proj = tsne.fit_transform(emb_nasn)
# Plot those points as a scatter plot and label them based on the pred labels
cmap = cm.get_cmap('tab20')
fig, ax = plt.subplots(figsize=(8,8))
num_categories = 12
y = preprocessing(df)[1]

for lab in range(1,num_categories+1):
    indices = y==lab
    ax.scatter(tsne_proj[indices[:,0],0],tsne_proj[indices[:,0],1], c=np.array(cmap(lab)).reshape(1,4), label = lab ,alpha=1)
ax.legend(fontsize='large', markerscale=2)
plt.show()

model.layers

In [27]:
print(emb_nasn.shape)

(144, 1056)


In [8]:
data_x, data_y = preprocessing(df)
x_test, y_test = 

In [10]:
print(data_x.shape)
print(data_y.shape)

(144, 256, 256, 3)
(144, 1)


In [13]:
history = model.fit(
    x=data_x,
    y=data_y,
    epochs=EPOCH_NR,
    batch_size=BATCH_SIZE,
    validation_data=(x_test, y_test),
    callbacks=[NeptuneLogger()],
)

NameError: name 'x_test' is not defined

In [ ]:
# Metryki
y_test_pred = np.asarray(model.predict(x_test))
y_test_pred_class = np.argmax(y_test_pred, axis=1)

run["test/f1"] = f1_score(y_test, y_test_pred_class, average="micro")

fig, ax = plt.subplots(figsize=(16, 12))
plot_confusion_matrix(y_test, y_test_pred_class, ax=ax)
run["diagnostic_charts"].log(neptune.types.File.as_image(fig))

fig, ax = plt.subplots(figsize=(16, 12))
plot_roc(y_test, y_test_pred, ax=ax)
run["diagnostic_charts"].log(neptune.types.File.as_image(fig))

In [ ]:
'''
model.save("my_model.h5")
run["model"].upload("my_model.h5")
'''

In [11]:
run.stop()

Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 1 operations to synchronize with Neptune. Do not kill this process.
All 1 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/common/colab-test-run/e/COL-39262
